In [2]:
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz -O images.tar.gz
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz -O annotations.tar.gz

!mkdir -p data/raw
!tar -xzf images.tar.gz -C data/raw
!tar -xzf annotations.tar.gz -C data/raw

--2025-12-05 04:34:16--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://thor.robots.ox.ac.uk/pets/images.tar.gz [following]
--2025-12-05 04:34:17--  https://thor.robots.ox.ac.uk/pets/images.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/octet-stream]
Saving to: ‘images.tar.gz’

images.tar.gz       100%[===================>] 755.23M  34.7MB/s    in 23s     

2025-12-05 04:34:40 (32.9 MB/s) - ‘images.tar.gz’ saved [791918971/791918971]

--2025-12-05 04:34:40--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
Resolving www.r

In [ ]:
import os
import shutil
from glob import glob

import numpy as np
import cv2
from skimage.feature import hog
from skimage import color

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [10]:
paths_cat = glob("data/train/cat/*.jpg")
paths_not = glob("data/train/not_cat/*.jpg")

train_cat, val_cat = train_test_split(paths_cat, test_size=0.2, random_state=42)
train_not, val_not = train_test_split(paths_not, test_size=0.2, random_state=42)

os.makedirs("data2/train/cat", exist_ok=True)
os.makedirs("data2/train/not_cat", exist_ok=True)
os.makedirs("data2/val/cat", exist_ok=True)
os.makedirs("data2/val/not_cat", exist_ok=True)

for p in train_cat:
    shutil.copy(p, "data2/train/cat")
for p in val_cat:
    shutil.copy(p, "data2/val/cat")

for p in train_not:
    shutil.copy(p, "data2/train/not_cat")
for p in val_not:
    shutil.copy(p, "data2/val/not_cat")

print("Gatos:", cat_count)
print("No gatos:", not_cat_count)
print("Total:", cat_count + not_cat_count)

Gatos: 2399
No gatos: 4990
Total: 7389


In [11]:
def extract_hog(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128, 128))
    img_gray = color.rgb2gray(img)
    features, _ = hog(img_gray, pixels_per_cell=(8,8),
                      cells_per_block=(2,2), visualize=True)
    return features

# Modelo SVM


### Convertir el dataset en X, y

Se extrae HOG de cada imagen y se arma un dataset para SVM.

In [5]:
def build_dataset(base_path="data/train"):
    X = []
    y = []

    cat_paths = glob(os.path.join(base_path, "cat", "*.jpg"))
    notcat_paths = glob(os.path.join(base_path, "not_cat", "*.jpg"))

    print("Total gatos:", len(cat_paths))
    print("Total no gatos:", len(notcat_paths))

    for img in cat_paths:
        feat = extract_hog(img)
        X.append(feat)
        y.append(1)

    # no gatos → label = 0
    for img in notcat_paths:
        feat = extract_hog(img)
        X.append(feat)
        y.append(0)

    return np.array(X), np.array(y)

### Separar en train/test

In [ ]:
X, y = build_dataset()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

Total gatos: 2400
Total no gatos: 4990


 ### Entrenar el SVM (modelo baseline)

In [ ]:
svm = LinearSVC()
svm.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [ ]:
y_pred = svm.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.75      0.76       998
           1       0.51      0.53      0.52       480

    accuracy                           0.68      1478
   macro avg       0.64      0.64      0.64      1478
weighted avg       0.69      0.68      0.68      1478

[[753 245]
 [225 255]]


# Modelo CNN

### Preparacion de la Data

In [12]:
IMG_SIZE = (160, 160)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   rotation_range=15,
                                   zoom_range=0.1)

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    "data2/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

val_gen = test_datagen.flow_from_directory(
    "data2/val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

print(train_gen.class_indices)

Found 5912 images belonging to 2 classes.
Found 1478 images belonging to 2 classes.
{'cat': 0, 'not_cat': 1}


### MobileNetV2

In [13]:
base = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)

base.trainable = False   # congelar

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### Modelo

In [14]:
model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

### Entrenamiento

In [15]:
EPOCHS = 5  # puedes subirlo a 10 si ves que no sobreentrena

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
185/185 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.8540 - loss: 0.3219 - val_accuracy: 0.9750 - val_loss: 0.0845
Epoch 2/5
185/185 ━━━━━━━━━━━━━━━━━━━━ 255s 1s/step - accuracy: 0.9688 - loss: 0.0887 - val_accuracy: 0.9797 - val_loss: 0.0670
Epoch 3/5
185/185 ━━━━━━━━━━━━━━━━━━━━ 209s 1s/step - accuracy: 0.9736 - loss: 0.0697 - val_accuracy: 0.9811 - val_loss: 0.0602
Epoch 4/5
185/185 ━━━━━━━━━━━━━━━━━━━━ 211s 1s/step - accuracy: 0.9795 - loss: 0.0617 - val_accuracy: 0.9790 - val_loss: 0.0617
Epoch 5/5
185/185 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.9765 - loss: 0.0639 - val_accuracy: 0.9790 - val_loss: 0.0615


### Evaluacion

In [16]:
val_loss, val_acc = model.evaluate(val_gen)
print("Validation accuracy:", val_acc)

47/47 ━━━━━━━━━━━━━━━━━━━━ 43s 904ms/step - accuracy: 0.9749 - loss: 0.0649
Validation accuracy: 0.9790257215499878


### Predicciones

In [17]:
val_gen.reset()
preds = model.predict(val_gen)
pred_labels = (preds > 0.5).astype("int")
true_labels = val_gen.classes

print(classification_report(true_labels, pred_labels))
print(confusion_matrix(true_labels, pred_labels))

47/47 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step
              precision    recall  f1-score   support

           0       0.33      0.31      0.32       480
           1       0.68      0.69      0.68       998

    accuracy                           0.57      1478
   macro avg       0.50      0.50      0.50      1478
weighted avg       0.56      0.57      0.57      1478

[[151 329]
 [310 688]]


### Fine Tunning

In [ ]:
base.trainable = True

for layer in base.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history_ft = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=3
)

Epoch 1/3
185/185 ━━━━━━━━━━━━━━━━━━━━ 351s 2s/step - accuracy: 0.9502 - loss: 0.1167 - val_accuracy: 0.9702 - val_loss: 0.1124
Epoch 2/3
 22/185 ━━━━━━━━━━━━━━━━━━━━ 4:03 1s/step - accuracy: 0.9910 - loss: 0.0308

Se realizó una comparacion entre un modelo SVM y un modelo CNN. Por el momento se continua realizando diferentes pruebas y entendimiento de la data para obtener mejores resultados